# 💰 CAS Financial Literacy - Interactive Analysis

## IB CAS Project: Learn Stocks, Investing & Market Analysis with OpenBB

This notebook will help you:
- Understand how to fetch real market data
- Calculate technical indicators
- Visualize stock performance
- Analyze Bitcoin and other assets

---

## 📦 Step 1: Install Required Libraries

Run this cell first to install all necessary packages.

In [ ]:
# Install required packages (run once)
!pip install openbb plotly pandas numpy scipy yfinance -q
print("✅ All packages installed!")

## 📚 Step 2: Import Libraries

In [ ]:
# Import all required libraries
import warnings
warnings.filterwarnings('ignore')

from openbb import obb
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✅ Libraries imported successfully!")
print(f"📊 OpenBB version: {obb.__version__}")

---
## 📈 Part 1: Fetching Stock Data

Let's start by fetching data for a popular stock. You can change the symbol to any stock you want!

In [ ]:
# 🔧 CHANGE THIS to analyze different stocks!
STOCK_SYMBOL = "AAPL"  # Try: MSFT, TSLA, GOOGL, AMZN, META, NVDA

# Fetch 1 year of historical data
print(f"📊 Fetching data for {STOCK_SYMBOL}...")

stock_data = obb.equity.price.historical(
    symbol=STOCK_SYMBOL,
    provider="yfinance",
    start_date=(datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
).to_df()

print(f"✅ Fetched {len(stock_data)} days of data for {STOCK_SYMBOL}")
print(f"\n📅 Date Range: {stock_data.index[0].strftime('%Y-%m-%d')} to {stock_data.index[-1].strftime('%Y-%m-%d')}")

# Show the last 5 rows
print("\n📋 Latest Data:")
stock_data.tail()

### Understanding the Data

Each row represents one day of trading with:
- **Open**: Price at market open
- **High**: Highest price during the day
- **Low**: Lowest price during the day
- **Close**: Price at market close
- **Volume**: Number of shares traded

In [ ]:
# Basic statistics about the stock
current_price = stock_data['close'].iloc[-1]
year_high = stock_data['high'].max()
year_low = stock_data['low'].min()
avg_volume = stock_data['volume'].mean()

print(f"📊 {STOCK_SYMBOL} Statistics:")
print(f"   💵 Current Price: ${current_price:,.2f}")
print(f"   📈 52-Week High: ${year_high:,.2f}")
print(f"   📉 52-Week Low: ${year_low:,.2f}")
print(f"   📊 Average Volume: {avg_volume:,.0f} shares/day")

# Calculate year-to-date return
ytd_return = ((current_price - stock_data['close'].iloc[0]) / stock_data['close'].iloc[0]) * 100
print(f"   {'🟢' if ytd_return >= 0 else '🔴'} 1-Year Return: {ytd_return:+.2f}%")

---
## 📐 Part 2: Calculating Technical Indicators

Technical indicators help us understand:
- **Trend**: Is the price going up or down?
- **Momentum**: How strong is the move?
- **Volatility**: How much is the price swinging?

In [ ]:
def calculate_indicators(df):
    """Calculate all technical indicators."""
    
    # Make a copy to avoid modifying original
    data = df.copy()
    
    # ====== MOVING AVERAGES ======
    # Simple Moving Averages (SMA)
    data['SMA_20'] = data['close'].rolling(window=20).mean()
    data['SMA_50'] = data['close'].rolling(window=50).mean()
    data['SMA_200'] = data['close'].rolling(window=200).mean()
    
    # Exponential Moving Average (EMA) - gives more weight to recent prices
    data['EMA_12'] = data['close'].ewm(span=12, adjust=False).mean()
    data['EMA_26'] = data['close'].ewm(span=26, adjust=False).mean()
    
    # ====== BOLLINGER BANDS ======
    data['BB_middle'] = data['close'].rolling(20).mean()
    data['BB_std'] = data['close'].rolling(20).std()
    data['BB_upper'] = data['BB_middle'] + (data['BB_std'] * 2)
    data['BB_lower'] = data['BB_middle'] - (data['BB_std'] * 2)
    data['BB_width'] = (data['BB_upper'] - data['BB_lower']) / data['BB_middle']
    
    # ====== RSI (Relative Strength Index) ======
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    
    # ====== MACD ======
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
    data['MACD_histogram'] = data['MACD'] - data['MACD_signal']
    
    # ====== ATR (Average True Range) - Volatility ======
    data['TR'] = np.maximum(
        data['high'] - data['low'],
        np.maximum(
            abs(data['high'] - data['close'].shift()),
            abs(data['low'] - data['close'].shift())
        )
    )
    data['ATR'] = data['TR'].rolling(14).mean()
    
    # ====== RETURNS ======
    data['daily_return'] = data['close'].pct_change()
    data['cumulative_return'] = (1 + data['daily_return']).cumprod() - 1
    
    return data

# Calculate all indicators
stock_data = calculate_indicators(stock_data)

print("✅ Technical indicators calculated!")
print("\n📊 Indicators added:")
print("   • Moving Averages (SMA 20, 50, 200)")
print("   • Bollinger Bands")
print("   • RSI (Relative Strength Index)")
print("   • MACD (Moving Average Convergence Divergence)")
print("   • ATR (Average True Range)")

In [ ]:
# Current indicator values
latest = stock_data.iloc[-1]

print(f"📊 Current Technical Analysis for {STOCK_SYMBOL}:")
print(f"\n💵 Price: ${latest['close']:,.2f}")

print(f"\n📈 Moving Averages:")
print(f"   SMA 20: ${latest['SMA_20']:,.2f} {'🟢 Above' if latest['close'] > latest['SMA_20'] else '🔴 Below'}")
print(f"   SMA 50: ${latest['SMA_50']:,.2f} {'🟢 Above' if latest['close'] > latest['SMA_50'] else '🔴 Below'}")
if not pd.isna(latest['SMA_200']):
    print(f"   SMA 200: ${latest['SMA_200']:,.2f} {'🟢 Above' if latest['close'] > latest['SMA_200'] else '🔴 Below'}")

print(f"\n📉 Momentum Indicators:")
rsi = latest['RSI']
rsi_status = '🔴 Overbought' if rsi > 70 else '🟢 Oversold' if rsi < 30 else '⚪ Neutral'
print(f"   RSI (14): {rsi:.2f} - {rsi_status}")

macd_signal = '🟢 Bullish' if latest['MACD'] > latest['MACD_signal'] else '🔴 Bearish'
print(f"   MACD: {latest['MACD']:.4f} ({macd_signal})")

print(f"\n📊 Volatility:")
print(f"   ATR (14): ${latest['ATR']:.2f}")
print(f"   Bollinger Band Width: {latest['BB_width']:.4f}")

---
## 📊 Part 3: Visualizing the Data

Let's create a professional-looking dashboard with interactive charts!

In [ ]:
def create_technical_dashboard(data, symbol):
    """Create a comprehensive technical analysis dashboard."""
    
    # Create subplots
    fig = make_subplots(
        rows=5, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.4, 0.15, 0.15, 0.15, 0.15],
        subplot_titles=(
            f'{symbol} Price Action with Bollinger Bands',
            'Volume',
            'RSI (Relative Strength Index)',
            'MACD',
            'ATR (Volatility)'
        )
    )
    
    # ===== ROW 1: CANDLESTICK + MOVING AVERAGES + BOLLINGER BANDS =====
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        name=symbol,
        increasing_line_color='#00ff41',
        decreasing_line_color='#ff4444'
    ), row=1, col=1)
    
    # Moving Averages
    fig.add_trace(go.Scatter(
        x=data.index, y=data['SMA_20'],
        name='SMA 20', line=dict(color='cyan', width=1)
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=data.index, y=data['SMA_50'],
        name='SMA 50', line=dict(color='yellow', width=1)
    ), row=1, col=1)
    
    if not data['SMA_200'].isna().all():
        fig.add_trace(go.Scatter(
            x=data.index, y=data['SMA_200'],
            name='SMA 200', line=dict(color='red', width=2)
        ), row=1, col=1)
    
    # Bollinger Bands
    fig.add_trace(go.Scatter(
        x=data.index, y=data['BB_upper'],
        name='BB Upper', line=dict(color='gray', dash='dash', width=1)
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=data.index, y=data['BB_lower'],
        name='BB Lower', line=dict(color='gray', dash='dash', width=1),
        fill='tonexty', fillcolor='rgba(128,128,128,0.1)'
    ), row=1, col=1)
    
    # ===== ROW 2: VOLUME =====
    colors = ['#00ff41' if data['close'].iloc[i] >= data['open'].iloc[i] 
              else '#ff4444' for i in range(len(data))]
    
    fig.add_trace(go.Bar(
        x=data.index, y=data['volume'],
        name='Volume', marker_color=colors, opacity=0.7
    ), row=2, col=1)
    
    # ===== ROW 3: RSI =====
    fig.add_trace(go.Scatter(
        x=data.index, y=data['RSI'],
        name='RSI', line=dict(color='orange', width=2)
    ), row=3, col=1)
    
    # RSI reference lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", opacity=0.5, row=3, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", opacity=0.5, row=3, col=1)
    fig.add_hline(y=50, line_dash="dot", line_color="gray", opacity=0.3, row=3, col=1)
    
    # ===== ROW 4: MACD =====
    fig.add_trace(go.Scatter(
        x=data.index, y=data['MACD'],
        name='MACD', line=dict(color='blue', width=1.5)
    ), row=4, col=1)
    
    fig.add_trace(go.Scatter(
        x=data.index, y=data['MACD_signal'],
        name='Signal Line', line=dict(color='red', width=1.5)
    ), row=4, col=1)
    
    # MACD Histogram
    colors_hist = ['#00ff41' if val >= 0 else '#ff4444' for val in data['MACD_histogram']]
    fig.add_trace(go.Bar(
        x=data.index, y=data['MACD_histogram'],
        name='MACD Histogram', marker_color=colors_hist, opacity=0.5
    ), row=4, col=1)
    
    # ===== ROW 5: ATR =====
    fig.add_trace(go.Scatter(
        x=data.index, y=data['ATR'],
        name='ATR', line=dict(color='purple', width=2),
        fill='tozeroy', fillcolor='rgba(128,0,128,0.2)'
    ), row=5, col=1)
    
    # ===== LAYOUT =====
    fig.update_layout(
        title=f'{symbol} - Complete Technical Analysis Dashboard',
        template='plotly_dark',
        height=1200,
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=3, col=1, range=[0, 100])
    fig.update_yaxes(title_text="MACD", row=4, col=1)
    fig.update_yaxes(title_text="ATR", row=5, col=1)
    
    return fig

# Create and display the dashboard
dashboard = create_technical_dashboard(stock_data, STOCK_SYMBOL)
dashboard.show()

---
## ₿ Part 4: Bitcoin Analysis

Let's apply the same analysis to Bitcoin!

In [ ]:
# Fetch Bitcoin data
print("₿ Fetching Bitcoin data...")

btc_data = obb.crypto.price.historical(
    symbol="BTCUSD",
    provider="yfinance",
    start_date=(datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
).to_df()

print(f"✅ Fetched {len(btc_data)} days of Bitcoin data")

# Calculate indicators
btc_data = calculate_indicators(btc_data)

# Show current stats
btc_latest = btc_data.iloc[-1]
btc_ytd = ((btc_latest['close'] - btc_data['close'].iloc[0]) / btc_data['close'].iloc[0]) * 100

print(f"\n₿ Bitcoin Statistics:")
print(f"   💵 Current Price: ${btc_latest['close']:,.2f}")
print(f"   📈 52-Week High: ${btc_data['high'].max():,.2f}")
print(f"   📉 52-Week Low: ${btc_data['low'].min():,.2f}")
print(f"   {'🟢' if btc_ytd >= 0 else '🔴'} 1-Year Return: {btc_ytd:+.2f}%")
print(f"   📊 RSI: {btc_latest['RSI']:.2f}")
print(f"   📈 MACD Signal: {'🟢 Bullish' if btc_latest['MACD'] > btc_latest['MACD_signal'] else '🔴 Bearish'}")

In [ ]:
# Create Bitcoin dashboard
btc_dashboard = create_technical_dashboard(btc_data, 'BTC/USD')
btc_dashboard.show()

---
## 📊 Part 5: Comparing Multiple Assets

Let's compare the performance of multiple stocks!

In [ ]:
# Stocks to compare
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

comparison_data = {}

print("📊 Fetching data for comparison...")
for symbol in symbols:
    try:
        data = obb.equity.price.historical(
            symbol=symbol,
            provider="yfinance",
            start_date=(datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
        ).to_df()
        comparison_data[symbol] = data
        print(f"   ✅ {symbol}")
    except Exception as e:
        print(f"   ❌ {symbol}: {e}")

print(f"\n✅ Loaded data for {len(comparison_data)} stocks")

In [ ]:
# Calculate normalized returns for comparison
fig = go.Figure()

colors = ['cyan', 'yellow', 'green', 'orange', 'red']

for i, (symbol, data) in enumerate(comparison_data.items()):
    # Normalize to start at 100
    normalized = (data['close'] / data['close'].iloc[0]) * 100
    
    fig.add_trace(go.Scatter(
        x=data.index,
        y=normalized,
        name=symbol,
        line=dict(color=colors[i % len(colors)], width=2)
    ))

fig.update_layout(
    title='Stock Performance Comparison (Normalized to 100)',
    template='plotly_dark',
    height=600,
    yaxis_title='Normalized Price (Starting = 100)',
    xaxis_title='Date',
    legend=dict(orientation="h", yanchor="bottom", y=1.02)
)

fig.add_hline(y=100, line_dash="dash", line_color="white", opacity=0.3)

fig.show()

In [ ]:
# Performance summary table
print("📊 Performance Summary:")
print("=" * 60)

summary_data = []
for symbol, data in comparison_data.items():
    current = data['close'].iloc[-1]
    start = data['close'].iloc[0]
    ret = ((current - start) / start) * 100
    volatility = data['close'].pct_change().std() * np.sqrt(252) * 100
    
    summary_data.append({
        'Symbol': symbol,
        'Price': current,
        'Return': ret,
        'Volatility': volatility
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Return', ascending=False)

for _, row in summary_df.iterrows():
    emoji = '🟢' if row['Return'] >= 0 else '🔴'
    print(f"{emoji} {row['Symbol']:6} | Price: ${row['Price']:>10,.2f} | Return: {row['Return']:>+7.2f}% | Volatility: {row['Volatility']:.1f}%")

---
## 🎯 Part 6: Trading Signals Analysis

Let's identify potential trading signals based on technical indicators.

In [ ]:
def analyze_trading_signals(data, symbol):
    """Analyze current trading signals."""
    latest = data.iloc[-1]
    prev = data.iloc[-2]
    
    signals = []
    
    # Trend signals (Moving Averages)
    if latest['close'] > latest['SMA_20'] > latest['SMA_50']:
        signals.append(('🟢', 'Strong Uptrend', 'Price above SMA 20 and SMA 50'))
    elif latest['close'] < latest['SMA_20'] < latest['SMA_50']:
        signals.append(('🔴', 'Strong Downtrend', 'Price below SMA 20 and SMA 50'))
    else:
        signals.append(('⚪', 'Mixed Trend', 'No clear trend direction'))
    
    # Golden Cross / Death Cross
    if not pd.isna(latest['SMA_200']) and not pd.isna(prev['SMA_200']):
        if latest['SMA_50'] > latest['SMA_200'] and prev['SMA_50'] <= prev['SMA_200']:
            signals.append(('🟢', 'GOLDEN CROSS!', 'SMA 50 crossed above SMA 200 - Strong buy signal'))
        elif latest['SMA_50'] < latest['SMA_200'] and prev['SMA_50'] >= prev['SMA_200']:
            signals.append(('🔴', 'DEATH CROSS!', 'SMA 50 crossed below SMA 200 - Strong sell signal'))
    
    # RSI signals
    if latest['RSI'] > 70:
        signals.append(('🔴', 'RSI Overbought', f'RSI at {latest["RSI"]:.1f} - Consider taking profits'))
    elif latest['RSI'] < 30:
        signals.append(('🟢', 'RSI Oversold', f'RSI at {latest["RSI"]:.1f} - Potential buying opportunity'))
    elif 30 <= latest['RSI'] <= 70:
        signals.append(('⚪', 'RSI Neutral', f'RSI at {latest["RSI"]:.1f} - No extreme conditions'))
    
    # MACD signals
    if latest['MACD'] > latest['MACD_signal'] and prev['MACD'] <= prev['MACD_signal']:
        signals.append(('🟢', 'MACD Bullish Cross', 'MACD crossed above signal line'))
    elif latest['MACD'] < latest['MACD_signal'] and prev['MACD'] >= prev['MACD_signal']:
        signals.append(('🔴', 'MACD Bearish Cross', 'MACD crossed below signal line'))
    elif latest['MACD'] > latest['MACD_signal']:
        signals.append(('🟢', 'MACD Bullish', 'MACD above signal line'))
    else:
        signals.append(('🔴', 'MACD Bearish', 'MACD below signal line'))
    
    # Bollinger Band signals
    if latest['close'] >= latest['BB_upper']:
        signals.append(('🔴', 'At Upper Bollinger Band', 'Price at upper band - Potentially overbought'))
    elif latest['close'] <= latest['BB_lower']:
        signals.append(('🟢', 'At Lower Bollinger Band', 'Price at lower band - Potentially oversold'))
    
    return signals

# Analyze signals for our stock
signals = analyze_trading_signals(stock_data, STOCK_SYMBOL)

print(f"🎯 Trading Signals for {STOCK_SYMBOL}:")
print("=" * 60)
for emoji, signal, explanation in signals:
    print(f"{emoji} {signal}")
    print(f"   └─ {explanation}")
    print()

# Overall sentiment
bullish = sum(1 for e, _, _ in signals if e == '🟢')
bearish = sum(1 for e, _, _ in signals if e == '🔴')
total = len(signals)

print("=" * 60)
print(f"📊 Signal Summary: {bullish} Bullish | {bearish} Bearish | {total - bullish - bearish} Neutral")

if bullish > bearish:
    print(f"🎯 Overall Sentiment: 🟢 BULLISH")
elif bearish > bullish:
    print(f"🎯 Overall Sentiment: 🔴 BEARISH")
else:
    print(f"🎯 Overall Sentiment: ⚪ NEUTRAL")

---
## 📝 Part 7: Try It Yourself!

Now it's your turn! Modify the code below to analyze any stock or crypto you're interested in.

In [ ]:
# =====================================================
# 🎮 YOUR TURN: ANALYZE YOUR OWN STOCK!
# =====================================================

# Change this to any stock or crypto symbol
MY_SYMBOL = "NVDA"  # Try: NVDA, AMD, META, NFLX, DIS, COIN, etc.

# Set True for crypto, False for stocks
IS_CRYPTO = False

# Fetch the data
print(f"📊 Analyzing {MY_SYMBOL}...")

if IS_CRYPTO:
    my_data = obb.crypto.price.historical(
        symbol=f"{MY_SYMBOL}USD",
        provider="yfinance",
        start_date=(datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
    ).to_df()
else:
    my_data = obb.equity.price.historical(
        symbol=MY_SYMBOL,
        provider="yfinance",
        start_date=(datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
    ).to_df()

# Calculate indicators
my_data = calculate_indicators(my_data)

# Show analysis
my_latest = my_data.iloc[-1]
my_return = ((my_latest['close'] - my_data['close'].iloc[0]) / my_data['close'].iloc[0]) * 100

print(f"\n📊 {MY_SYMBOL} Analysis:")
print(f"   💵 Price: ${my_latest['close']:,.2f}")
print(f"   {'🟢' if my_return >= 0 else '🔴'} 1-Year Return: {my_return:+.2f}%")
print(f"   📈 RSI: {my_latest['RSI']:.2f}")
print(f"   📊 MACD: {'Bullish' if my_latest['MACD'] > my_latest['MACD_signal'] else 'Bearish'}")

# Analyze signals
my_signals = analyze_trading_signals(my_data, MY_SYMBOL)
print(f"\n🎯 Signals:")
for emoji, signal, _ in my_signals:
    print(f"   {emoji} {signal}")

In [ ]:
# Create dashboard for your symbol
my_dashboard = create_technical_dashboard(my_data, MY_SYMBOL)
my_dashboard.show()

---
## 🎓 Key Takeaways

### What You Learned:

1. **Fetching Real Market Data** - Using OpenBB to get stock and crypto prices

2. **Technical Indicators**:
   - **Moving Averages (SMA/EMA)** - Identify trends
   - **RSI** - Measure momentum (overbought/oversold)
   - **MACD** - Trend direction and momentum
   - **Bollinger Bands** - Volatility and potential reversals
   - **ATR** - Measure market volatility

3. **Chart Patterns** - Reading candlestick charts

4. **Trading Signals** - How to interpret indicators together

### Remember:

⚠️ **Technical analysis is not 100% accurate** - Always do your research!

⚠️ **Past performance doesn't guarantee future results**

⚠️ **Never invest money you can't afford to lose**

---

*Happy Learning! 📈*